In [1]:
# Setup
import importlib
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt

from sccoda.util import comp_ana as mod
from sccoda.util import cell_composition_data as dat
from sccoda.util import data_visualization as viz

import sccoda.datasets as scd

2025-07-18 01:30:33.827565: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-18 01:30:33.845199: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 01:30:33.913875: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 01:30:33.927476: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752816633.996027  811211 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752816634.00

In [ ]:
import pandas as pd
import scanpy as sc
cell_counts = pd.read_csv("../../data/celltype_counts_by_batch_disease.csv")  # columns: cell_id, disease, batch, celltype

data_all = dat.from_pandas(cell_counts, covariate_columns=["batch", "disease"])

print(data_all)

AnnData object with n_obs × n_vars = 36 × 8
    obs: 'batch', 'disease'


In [3]:
data_salm = data_all
model_salm = mod.CompositionalAnalysis(data_salm, formula="disease + batch", reference_cell_type="automatic")

Automatic reference selection! Reference cell type set to astrocytes
Zero counts encountered in data! Added a pseudocount of 0.5.


2025-07-18 01:33:39.144674: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [4]:
sim_results = model_salm.sample_hmc()

I0000 00:00:1752816823.187879  853621 service.cc:152] XLA service 0x14bae80076e0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752816823.187935  853621 service.cc:160]   StreamExecutor device (0): Host, Default Version
2025-07-18 01:33:43.306119: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1752816824.198123  853621 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
100%|██████████| 20000/20000 [02:41<00:00, 124.09it/s]


MCMC sampling finished. (205.017 sec)
Acceptance rate: 69.4%


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)
sim_results.set_fdr(est_fdr=0.05)
sim_results.summary()

In [ ]:
# saving
path = "all_ct_scCODA" 
sim_results.save(path) # output also copied into a easily readible format in ./all_ct_scCODA_0.05.out
# results of a second run is stored in ./all_ct_scCODA_0.05_2.out